In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('./data/train.csv')
test  = pd.read_csv('./data/test.csv')
sub   = pd.read_csv('./data/sample_submission.csv')
ag = pd.read_csv('./data/age_gender_info.csv')

# 임대보증금과 임대료는 '-'을 null로 바꾼 후 float로 타입 변경

train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)

train.loc[train.임대료=='-', '임대료'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
train['임대료'] = train['임대료'].astype(float)
test['임대료'] = test['임대료'].astype(float)

# test 자격유형 결측치 - 'C2411', 'C2253'

test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'
test.loc[test['단지코드'].isin(['C2253']) & test['자격유형'].isnull(), '자격유형'] = 'C'

# train, test 지하철 버스 결측치

bus_subway = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수']
test[bus_subway] = test[bus_subway].fillna(0)
train[bus_subway] = train[bus_subway].fillna(0)

all_ = pd.concat([train, test])
all2 = all_ 

# 임대료와 임대보증금이 결측치인 데이터와 아닌 데이터 분리


# 결측치가 있는 데이터 만들기

train_nc = (~train.isnull()).sum(axis=1)
test_nc = (~test.isnull()).sum(axis=1)

train_nc_rows = train[((train_nc==14) | (train_nc==13)) ]
test_nc_rows = test[((test_nc==13) | (test_nc==12)) ]

all_null = pd.concat([train_nc_rows, test_nc_rows])


# 결측치가 없는 데이터 만들기

train_ic_rows = train[((train_nc==15))]
test_ic_rows = test[((test_nc==14)) ]

all_is = pd.concat([train_ic_rows, test_ic_rows])

In [2]:
all_.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [3]:
all_is.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [4]:
all_null.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [5]:
cols = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
        '자격유형','도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수', '단지내주차면수']

### 임대보증금과 임대료 중 임대보증금을 먼저 구한 후 임대료를 나중에 구하기(임대보증금이 더 중요한다고 판단)
### 임대보증금 구하기

## Catboost 사용

In [6]:
all_is.head(1)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,1015.0


In [7]:
cat_features = ['단지코드', '임대건물구분', '지역', '공급유형', '자격유형' ]
target = '임대보증금'

In [8]:
feature_names = cols 

In [9]:
x_train = all_is[cols]
y_train = all_is[target]
x_test = all_null[cols]

In [10]:
from catboost import CatBoostRegressor

catb_model = CatBoostRegressor(
                               loss_function='MAE',
                               n_estimators = 1000, 
                               learning_rate = 0.03, 
                               random_state = 10
                               )
catb_model.fit(x_train, y_train, cat_features, verbose=False, plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### 임대보증금 예측

In [11]:
predictions = catb_model.predict(x_test)
predictions[0:10]

array([18658159.2562555 , 18658159.2562555 , 18658159.2562555 ,
       41914027.34453456, 11333318.44663431, 11344324.01314857,
       11344324.01314857, 13579192.70878804, 13579192.70878804,
       13579192.70878804])

In [12]:
all_null['임대보증금']

88     NaN
89     NaN
90     NaN
91     NaN
101    NaN
        ..
1006   NaN
1014   NaN
1015   NaN
1016   NaN
1017   NaN
Name: 임대보증금, Length: 776, dtype: float64

In [13]:
all_null['임대보증금'] = predictions
all_null['임대보증금']

88      1.865816e+07
89      1.865816e+07
90      1.865816e+07
91      4.191403e+07
101     1.133332e+07
            ...     
1006    8.111617e+06
1014    1.780529e+07
1015    2.408586e+07
1016    2.413168e+07
1017    3.122764e+07
Name: 임대보증금, Length: 776, dtype: float64

In [14]:
all_null['임대보증금'].isnull().sum()

0

### 임대료 차례

In [16]:
# 임대보증금을 추가한다.

cols = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '임대보증금',

        '자격유형','도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수', '단지내주차면수']

In [17]:
cat_features = ['단지코드', '임대건물구분', '지역', '공급유형', '자격유형' ]
target = '임대료'

In [18]:
x_train = all_is[cols]
y_train = all_is[target]
x_test = all_null[cols]

In [19]:
from catboost import CatBoostRegressor

catb_model = CatBoostRegressor(
                               loss_function='MAE',
                               n_estimators = 1000, 
                               learning_rate = 0.03, 
                               random_state = 10
                               )
catb_model.fit(x_train, y_train, cat_features, verbose=False, plot = False)

In [20]:
predictions = catb_model.predict(x_test)
predictions[0:10]

array([115687.2444771 , 115687.2444771 , 115687.2444771 , 214028.09317782,
        71700.86436456,  72996.12057575,  72996.12057575,  74796.40696019,
        74796.40696019,  74796.40696019])

In [21]:
all_null['임대료']

88     NaN
89     NaN
90     NaN
91     NaN
101    NaN
        ..
1006   NaN
1014   NaN
1015   NaN
1016   NaN
1017   NaN
Name: 임대료, Length: 776, dtype: float64

In [22]:
all_null['임대료'] = predictions
all_null['임대료']

88      115687.244477
89      115687.244477
90      115687.244477
91      214028.093178
101      71700.864365
            ...      
1006    107366.224931
1014     91789.761026
1015    130213.666221
1016    130493.203902
1017    158172.724712
Name: 임대료, Length: 776, dtype: float64

In [23]:
all_null['임대료'].isnull().sum()

0

### 다시 Train 과 Test로 나누기

In [24]:
new_all = pd.concat([all_is, all_null])
new_all

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,1.566700e+07,103680.000000,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,1.566700e+07,103680.000000,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,2.730400e+07,184330.000000,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,2.730400e+07,184330.000000,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,2.730400e+07,184330.000000,0.0,3.0,1425.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006,C2152,120,아파트,강원도,영구임대,33.84,54,9.0,C,8.111617e+06,107366.224931,0.0,1.0,40.0,NaN
1014,C1267,675,아파트,경상남도,행복주택,16.94,50,38.0,L,1.780529e+07,91789.761026,0.0,1.0,467.0,NaN
1015,C1267,675,아파트,경상남도,행복주택,26.85,66,38.0,L,2.408586e+07,130213.666221,0.0,1.0,467.0,NaN
1016,C1267,675,아파트,경상남도,행복주택,26.85,8,38.0,L,2.413168e+07,130493.203902,0.0,1.0,467.0,NaN


In [25]:
new_test = new_all[new_all['등록차량수'].isnull()]
print(new_test.shape)
new_test.head()

(1022, 15)


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000.0,189840.0,0.0,2.0,683.0,NaN
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000.0,249930.0,0.0,2.0,683.0,NaN
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000.0,249930.0,0.0,2.0,683.0,NaN
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000.0,249930.0,0.0,2.0,683.0,NaN
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000.0,296780.0,0.0,2.0,683.0,NaN


In [26]:
new_train = new_all[new_all['등록차량수'].notnull()]
print(new_train.shape)
new_train.head()

(2952, 15)


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0


### 모든 결측치를 채운 뒤 등록차량수 예측해보기

In [27]:
cols = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '임대보증금', '임대료',

        '자격유형','도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수', '단지내주차면수']

cat_features = ['단지코드', '임대건물구분', '지역', '공급유형', '자격유형' ]
target = '등록차량수'

x_train = new_train[cols]
y_train = new_train[target]
x_test = new_test[cols]

In [28]:
from catboost import CatBoostRegressor

catb_model = CatBoostRegressor(
                               loss_function='MAE',
                               n_estimators = 1000, 
                               learning_rate = 0.03, 
                               random_state = 10
                               )
catb_model.fit(x_train, y_train, cat_features, verbose=False, plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [29]:
predictions = catb_model.predict(x_test)
new_test['등록차량수'] = predictions
new_test['등록차량수']

<ipython-input-29-5109ef92bda5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test['등록차량수'] = predictions


0       667.216894
1       711.259556
2       701.279384
3       701.279384
4       687.752702
           ...    
1006     15.589040
1014    388.371964
1015    394.393964
1016    384.001541
1017    412.421535
Name: 등록차량수, Length: 1022, dtype: float64

In [30]:
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission.head()

,code,num
0,C1072,0
1,C1128,0
2,C1456,0
3,C1840,0
4,C1332,0


In [31]:
sample_submission['num'] = new_test['등록차량수'] 
sample_submission.head()

,code,num
0,C1072,667.216894
1,C1128,711.259556
2,C1456,701.279384
3,C1840,701.279384
4,C1332,687.752702


### 제출

In [32]:
# 214등 112.05811

sample_submission.to_csv('notnull_catboost_baseline_20210724.csv', index=False)

### 개인연습

### 1.구분값 변경하기
####  - 임대건물구분, 지역코드, 공급유형, 자격유형

### 2. 주요 변수로만 모델 돌리기
####  - corr()가 높은 것만

In [55]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

new_train['임대건물구분'] = encoder.fit_transform(new_train['임대건물구분'].values)
new_train['지역'] = encoder.fit_transform(new_train['지역'].values)
new_train['공급유형'] = encoder.fit_transform(new_train['공급유형'].values)
new_train['자격유형'] = encoder.fit_transform(new_train['자격유형'].values)

new_test['임대건물구분'] = encoder.fit_transform(new_test['임대건물구분'].values)
new_test['지역'] = encoder.fit_transform(new_test['지역'].values)
new_test['공급유형'] = encoder.fit_transform(new_test['공급유형'].values)
new_test['자격유형'] = encoder.fit_transform(new_test['자격유형'].values)

<ipython-input-55-5649dcbdedae>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train['임대건물구분'] = encoder.fit_transform(new_train['임대건물구분'].values)
<ipython-input-55-5649dcbdedae>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train['지역'] = encoder.fit_transform(new_train['지역'].values)
<ipython-input-55-5649dcbdedae>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [56]:
new_train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,1,3,5,39.72,134,38.0,0,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
1,C2483,900,1,3,5,39.72,15,38.0,0,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
2,C2483,900,1,3,5,51.93,385,38.0,0,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
3,C2483,900,1,3,5,51.93,15,38.0,0,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
4,C2483,900,1,3,5,51.93,41,38.0,0,27304000.0,184330.0,0.0,3.0,1425.0,1015.0


In [39]:
new_train.corr()

,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
총세대수,1.000000,-0.340668,-0.057719,0.113577,0.042271,0.098205,0.068192,0.166746,0.081464,0.017944,0.209509,-0.000295,0.501488,0.316703
임대건물구분,-0.340668,1.000000,-0.109903,-0.439494,-0.009347,0.372043,0.242644,-0.164922,0.081176,0.222996,-0.162156,0.009191,0.408489,0.449189
지역,-0.057719,-0.109903,1.000000,0.012053,-0.006767,-0.043860,-0.015326,-0.034536,-0.105042,-0.124188,-0.055265,-0.077710,-0.153685,-0.169999
공급유형,0.113577,-0.439494,0.012053,1.000000,-0.235806,-0.191230,0.143722,0.632138,-0.205365,-0.634368,0.087256,-0.031331,-0.367753,-0.518342
전용면적,0.042271,-0.009347,-0.006767,-0.235806,1.000000,-0.035479,-0.089201,-0.174906,0.369121,0.344714,-0.011591,-0.010087,0.071260,0.115453
전용면적별세대수,0.098205,0.372043,-0.043860,-0.191230,-0.035479,1.000000,0.166472,-0.072178,-0.061792,0.031078,-0.047729,0.046448,0.281989,0.247085
공가수,0.068192,0.242644,-0.015326,0.143722,-0.089201,0.166472,1.000000,0.205525,0.041422,-0.118950,-0.076314,0.042862,0.289819,0.118121
자격유형,0.166746,-0.164922,-0.034536,0.632138,-0.174906,-0.072178,0.205525,1.000000,0.089829,-0.218956,0.005772,-0.025270,-0.070605,-0.154491
임대보증금,0.081464,0.081176,-0.105042,-0.205365,0.369121,-0.061792,0.041422,0.089829,1.000000,0.567108,0.018021,0.107980,0.280342,0.322956
임대료,0.017944,0.222996,-0.124188,-0.634368,0.344714,0.031078,-0.118950,-0.218956,0.567108,1.000000,0.019137,0.028994,0.334683,0.448715


### 등록차량수와 연관성이 높은 변수 순위
#### - 단지내주차면수, 임대건물구분, 임대료, 임대보증금, 총세대수, 전용면적별세대수, 공가수, 전용면적, 버스정류장수 순(0이상)

### 주요 변수로만 모델 예측 돌리기

In [57]:
cols = ['총세대수', '임대건물구분', '전용면적', '전용면적별세대수', '공가수', '임대보증금', '임대료',

        '도보 10분거리 내 버스정류장 수', '단지내주차면수']

target = '등록차량수'

x_train = new_train[cols]
y_train = new_train[target]
x_test = new_test[cols]

In [41]:
from catboost import CatBoostRegressor

catb_model = CatBoostRegressor(
                               loss_function='MAE',
                               n_estimators = 1000, 
                               learning_rate = 0.03, 
                               random_state = 10
                               )
catb_model.fit(x_train, y_train, verbose=False, plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [58]:
x_test.head()

,총세대수,임대건물구분,전용면적,전용면적별세대수,공가수,임대보증금,임대료,도보 10분거리 내 버스정류장 수,단지내주차면수
0,754,1,39.79,116,14.0,22830000.0,189840.0,2.0,683.0
1,754,1,46.81,30,14.0,36048000.0,249930.0,2.0,683.0
2,754,1,46.90,112,14.0,36048000.0,249930.0,2.0,683.0
3,754,1,46.90,120,14.0,36048000.0,249930.0,2.0,683.0
4,754,1,51.46,60,14.0,43497000.0,296780.0,2.0,683.0


In [59]:
new_test['등록차량수'] = predictions
new_test['등록차량수']

<ipython-input-59-95c60ba04fbe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test['등록차량수'] = predictions


0       667.216894
1       711.259556
2       701.279384
3       701.279384
4       687.752702
           ...    
1006     15.589040
1014    388.371964
1015    394.393964
1016    384.001541
1017    412.421535
Name: 등록차량수, Length: 1022, dtype: float64

In [67]:
predictions = catb_model.predict(x_test)
new_test['등록차량수'] = predictions

###반올림 적용
new_test['등록차량수'] = new_test['등록차량수'].round(0)
new_test['등록차량수']

<ipython-input-67-db72272f8219>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test['등록차량수'] = predictions
<ipython-input-67-db72272f8219>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test['등록차량수'] = new_test['등록차량수'].round(0)


0       664.0
1       710.0
2       712.0
3       712.0
4       710.0
        ...  
1006     46.0
1014    372.0
1015    408.0
1016    387.0
1017    411.0
Name: 등록차량수, Length: 1022, dtype: float64

In [68]:
sample_submission['num'] = new_test['등록차량수'] 
sample_submission.head()

,code,num
0,C1072,664.0
1,C1128,710.0
2,C1456,712.0
3,C1840,712.0
4,C1332,710.0


In [69]:
sample_submission.to_csv('notnull_catboost_baseline_20210730.csv', index=False)